In [1]:
import argparse
import os
import shlex
import sys
from pprint import pformat

import numpy as np
import pandas as pd
import tensorflow_addons as tfa
from tensorflow import keras

from augmentation import c as aug_levels
from lib.keras_tb_logger import TensorBoard_Logger, gpu_temp_logger, lr_logger
from lib.utils import (
    create_dir,
    fix_random_seed,
    read_json,
    save_keras_model,
    write_json,
)
from src.config import c
from src.generator import Generator
from src.models import Wave_Model_Builder
from src.services import get_msg_provider, get_wave_provider

In [2]:
mb = Wave_Model_Builder(
    n_classes=100,
    wave_len_samples=32000 * 5,
    wave_sample_rate=32000,
    n_fft=2048,
    spec_power=2,
    body="enb0",
    imagenet_weights=False,
    extra_dense_layers=None,
    dropout=None,
)

In [3]:
m = mb.build()

In [4]:
save_keras_model(m, "model.png", dpi=75, rankdir="LR")

In [5]:
for i, layer in enumerate(m.layers):
    print(layer.name)

i_wave
mel_spectrogram
power_to_db
float2d_to_rgb
efficientnetb0
i_year
i_month
i_lat
i_lon
f_msg
f_date
f_month_sincos
f_lat
f_lon_sincos
features
o_classes


In [7]:
'i_wave' in [layer.name for layer  in m.layers]

True